# Testing NCBI hg38.p14 GTF file

*Anusha Aggarwal*

In [42]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
import os
from pyfaidx import Fasta
from Bio.Seq import Seq

In [59]:
gtf_cds = pd.read_csv('/mnt/home/aaggarwal/ceph/gates_proj/ncbi_genome_hg38.p14/hg38.p14.ncbiRefSeq.CDS_final_gtf.csv')
gtf_trans = pd.read_csv('/mnt/home/aaggarwal/ceph/gates_proj/ncbi_genome_hg38.p14/hg38.p14.ncbiRefSeq.transcript_final_gtf.csv')

In [60]:
gtf_cds

,seqname,source,feature,start,end,score,strand,frame,attributes,gene_id,transcript_id,entrez_id,length
0,chr1,BestRefSeq,CDS,65565,65573,.,+,0,"gene_id ""OR4F5""; transcript_id ""NM_001005484.2...",OR4F5,NM_001005484.2,79501,9
1,chr1,BestRefSeq,CDS,69037,70005,.,+,0,"gene_id ""OR4F5""; transcript_id ""NM_001005484.2...",OR4F5,NM_001005484.2,79501,969
2,chr1,BestRefSeq,start_codon,65565,65567,.,+,0,"gene_id ""OR4F5""; transcript_id ""NM_001005484.2...",OR4F5,NM_001005484.2,79501,3
3,chr1,BestRefSeq,stop_codon,70006,70008,.,+,0,"gene_id ""OR4F5""; transcript_id ""NM_001005484.2...",OR4F5,NM_001005484.2,79501,3
4,chr1,BestRefSeq,CDS,450743,451678,.,-,0,"gene_id ""OR4F29""; transcript_id ""NM_001005221....",OR4F29,NM_001005221.2,729759,936
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229586,chrY,BestRefSeq,stop_codon,25038098,25038100,.,-,0,"gene_id ""BPY2C""; transcript_id ""NM_001002761.1...",BPY2C,NM_001002761.1,442868,3
229587,chrY,BestRefSeq,CDS,25622443,25624034,.,+,0,"gene_id ""CDY1""; transcript_id ""NM_004680.3""; d...",CDY1,NM_004680.3,9085,1592
229588,chrY,BestRefSeq,CDS,25624455,25624524,.,+,1,"gene_id ""CDY1""; transcript_id ""NM_004680.3""; d...",CDY1,NM_004680.3,9085,70
229589,chrY,BestRefSeq,start_codon,25622443,25622445,.,+,0,"gene_id ""CDY1""; transcript_id ""NM_004680.3""; d...",CDY1,NM_004680.3,9085,3


In [61]:
gtf_trans

,seqname,source,feature,start,end,score,strand,frame,attributes,gene_id,transcript_id,entrez_id
0,chr1,BestRefSeq,transcript,65419,71585,.,+,.,"gene_id ""OR4F5""; transcript_id ""NM_001005484.2...",OR4F5,NM_001005484.2,79501
1,chr1,BestRefSeq,transcript,450740,451678,.,-,.,"gene_id ""OR4F29""; transcript_id ""NM_001005221....",OR4F29,NM_001005221.2,729759
2,chr1,BestRefSeq,transcript,685716,686654,.,-,.,"gene_id ""OR4F16""; transcript_id ""NM_001005277....",OR4F16,NM_001005277.1,81399
3,chr1,BestRefSeq,transcript,923923,944574,.,+,.,"gene_id ""SAMD11""; transcript_id ""NM_001385641....",SAMD11,NM_001385641.1,148398
4,chr1,BestRefSeq,transcript,944203,959256,.,-,.,"gene_id ""NOC2L""; transcript_id ""NM_015658.4""; ...",NOC2L,NM_015658.4,26155
...,...,...,...,...,...,...,...,...,...,...,...,...
19161,chrY,BestRefSeq,transcript,24618004,24639207,.,+,.,"gene_id ""BPY2B""; transcript_id ""NM_001002760.1...",BPY2B,NM_001002760.1,442867
19162,chrY,BestRefSeq,transcript,24763069,24813393,.,-,.,"gene_id ""DAZ3""; transcript_id ""NM_020364.4""; d...",DAZ3,NM_020364.4,57054
19163,chrY,BestRefSeq,transcript,24833919,24907040,.,+,.,"gene_id ""DAZ4""; transcript_id ""NM_001388484.1""...",DAZ4,NM_001388484.1,57135
19164,chrY,BestRefSeq,transcript,25030901,25052104,.,-,.,"gene_id ""BPY2C""; transcript_id ""NM_001002761.1...",BPY2C,NM_001002761.1,442868


In [62]:
unique_gene_ids_count = gtf_cds["gene_id"].nunique()
unique_gene_ids_count

19166

In [63]:
unique_gene_ids_count = gtf_cds["entrez_id"].nunique()
unique_gene_ids_count

19166

In [213]:
def get_adjusted_cds_windows(transcript_id, chrom, strand, gtf_df):
    transcript_df = gtf_df[gtf_df['transcript_id'] == transcript_id]
    cds_df = transcript_df[transcript_df['feature'] == 'CDS']
    start_df = transcript_df[transcript_df['feature'] == 'start_codon']
    stop_df = transcript_df[transcript_df['feature'] == 'stop_codon']

    if cds_df.empty or start_df.empty or stop_df.empty:
        return None

    start_codon_start = start_df.iloc[0]['start']
    stop_codon_end = stop_df.iloc[0]['end']

    if strand == '+':
        cds_df = cds_df[
            (cds_df['end'] >= start_codon_start) &
            (cds_df['start'] <= stop_codon_end)
        ].sort_values(by='start')
    else:
        start_codon_end = start_df.iloc[0]['end']
        stop_codon_start = stop_df.iloc[0]['start']
    
        cds_df = cds_df[
            (cds_df['end'] >= stop_codon_start) &
            (cds_df['start'] <= start_codon_end)
        ].sort_values(by='end', ascending=False)

    if cds_df.empty:
        return None

    cds_windows = []
    for i, row in cds_df.iterrows():
        start = row['start']
        end = row['end']

        if strand == '+':
            if start <= start_codon_start <= end:
                start = start_codon_start
        else:
            if start <= start_codon_end <= end:
                end = start_codon_end

        if strand == '+':
            if start <= stop_codon_end <= end:
                end = stop_codon_end
        else:
            if start <= stop_codon_start <= end:
                start = stop_codon_start

        if end > start:
            cds_windows.append((start, end, int(row['frame'])))

    return cds_windows if cds_windows else None


In [232]:
gtf_cds[gtf_cds['entrez_id'] == 339456]

,seqname,source,feature,start,end,score,strand,frame,attributes,gene_id,transcript_id,entrez_id,length
539,chr1,BestRefSeq,CDS,1919182,1919265,.,-,0,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,84
540,chr1,BestRefSeq,CDS,1919045,1919088,.,-,0,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,44
541,chr1,BestRefSeq,CDS,1918893,1918934,.,-,1,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,42
542,chr1,BestRefSeq,CDS,1918253,1918431,.,-,1,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,179
543,chr1,BestRefSeq,CDS,1917885,1918162,.,-,2,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,278
544,chr1,BestRefSeq,start_codon,1919263,1919265,.,-,0,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,3
545,chr1,BestRefSeq,stop_codon,1917882,1917884,.,-,0,"gene_id ""TMEM52""; transcript_id ""NM_178545.4"";...",TMEM52,NM_178545.4,339456,3


In [216]:
gtf_cds[gtf_cds['transcript_id'] == 'NM_014188.3']

,seqname,source,feature,start,end,score,strand,frame,attributes,gene_id,transcript_id,entrez_id,length
415,chr1,BestRefSeq,CDS,1574478,1574557,.,-,0,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,80
416,chr1,BestRefSeq,CDS,1564773,1564916,.,-,1,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,144
417,chr1,BestRefSeq,CDS,1544863,1545002,.,-,1,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,140
418,chr1,BestRefSeq,CDS,1543869,1543987,.,-,2,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,119
419,chr1,BestRefSeq,CDS,1542069,1542167,.,-,0,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,99
420,chr1,BestRefSeq,start_codon,1574555,1574557,.,-,0,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,3
421,chr1,BestRefSeq,stop_codon,1542066,1542068,.,-,0,"gene_id ""SSU72""; transcript_id ""NM_014188.3""; ...",SSU72,NM_014188.3,29101,3


In [217]:
windows = get_adjusted_cds_windows("NM_014188.3", "chr1", "-", gtf_cds)

In [218]:
windows

[(1574478, 1574557, 0),
 (1564773, 1564916, 1),
 (1544863, 1545002, 1),
 (1543869, 1543987, 2),
 (1542069, 1542167, 0)]

In [220]:
genome_fasta_dir = "/mnt/home/aaggarwal/ceph/gates_proj/ncbi_genome_hg38.p14"

# load the genome
fasta_files = {
    f.replace(".fa", ""): os.path.join(genome_fasta_dir, f)
    for f in os.listdir(genome_fasta_dir) if f.endswith(".fa")
}
fasta_index = {k: Fasta(v) for k, v in fasta_files.items()}

In [221]:
cds_windows = sorted(windows, key=lambda x: x[0], reverse=True)
cds_windows

[(1574478, 1574557, 0),
 (1564773, 1564916, 1),
 (1544863, 1545002, 1),
 (1543869, 1543987, 2),
 (1542069, 1542167, 0)]

In [222]:
genome = fasta_index['chr1']

In [223]:
chrom = genome['chr1'] 

In [224]:
seq_parts = []

In [225]:
for i, (start, end, frame) in enumerate(cds_windows):
    start = int(start)
    end = int(end)
    frame = int(frame)

    exon_seq = chrom[start-1:end].seq.upper()

    if i == 0:
        exon_seq = exon_seq[frame:]

    seq_parts.append(exon_seq)

In [226]:
full_seq = ''.join(seq_parts[::-1])  # reverse exon order
full_seq_rc = str(Seq(full_seq).reverse_complement())

In [227]:
protein = str(Seq(full_seq_rc).translate(to_stop=False))

In [228]:
print(f"First codon: {full_seq_rc[:3]}")
print(f"Protein start: {protein[:10]}")
print(f"Full protein: {protein}")

First codon: ATG
Protein start: MPSSPLRVAV
Full protein: MPSSPLRVAVVCSSNQNRSMEAHNILSKRGFSVRSFGTGTHVKLPGPAPDKPNVYDFKTTYDQMYNDLLRKDKELYTQNGILHMLDRNKRIKPRPERFQNCKDLFDLILTCEERVYDQVVEDLNSREQETCQPVHVVNVDIQDNHEEATLGAFLICELCQCIQHTEDMENEIDELLQEFEEKSGRTFLHTVCFY


In [229]:
seq_parts = []
for i, (start, end, frame) in enumerate(cds_windows):
    start = int(start)
    end = int(end)
    frame = int(frame)

    exon_seq = chrom[start-1:end].seq.upper()

    exon_seq = exon_seq[frame:]

    seq_parts.append(exon_seq)

In [230]:
full_seq = ''.join(seq_parts[::-1])  # reverse exon order
full_seq_rc = str(Seq(full_seq).reverse_complement())

protein = str(Seq(full_seq_rc).translate(to_stop=False))

print(f"First codon: {full_seq_rc[:3]}")
print(f"Protein start: {protein[:10]}")
print(f"Full protein: {protein}")

First codon: ATG
Protein start: MPSSPLRVAV
Full protein: MPSSPLRVAVVCSSNQNRSMEAHNILSKRGFSVRSFGTGTHVKLPGPAPDKPNVYDFKTTYDQMYNDLLRKDKEPIHRMGFYICWTEIRESSPGQKDSRTAKTCLI*SSLAKRECMTRWWKSEFQRTGDLPACARGQCGHPGQPRGGHPGGVSHL*ALPVSSTRKTWRTRSTSCCRSSRRRVAAPFCTPSAS


/mnt/home/aaggarwal/miniconda3/envs/target-genes/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
